In [1]:
# default_exp convert_to_zarr

# Notebook which converts per region netCDF files to Zarr files to make them more efficient when indexing

### uses pangeo_small environment

In [1]:
#test_ignore
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import xarray as xr
import zarr
from joblib import Parallel, delayed
import pandas as pd
import os
from numcodecs import Blosc

## Set the parameters here
### Ensure all State and Regions you want to transform are specified here. 

In [3]:
#test_ignore
data_root = '/media/scottcha/Data2/OAPMLData/'
output_root = '/media/scottcha/E1/Data/OAPMLData/'
interpolation = 1

#currently only have Washington regions and one season specified for the tutorial
#uncomment regions and seasons if doing a larger transform
regions = {#'Utah': ['Abajos', 'Logan', 'Moab', 'Ogden', 'Provo', 
           #'Salt Lake', 'Skyline', 'Uintas'],  
           'Colorado': ['Grand Mesa Zone', 'Sangre de Cristo Range', 'Steamboat Zone', 'Front Range Zone',
           'Vail Summit Zone', 'Sawatch Zone', 'Aspen Zone', 
           'North San Juan Mountains', 'South San Juan Mountains', 'Gunnison Zone'],
           #'Washington': ['Mt Hood', 'Olympics', 'Snoqualmie Pass', 'Stevens Pass',
           #'WA Cascades East, Central', 'WA Cascades East, North', 'WA Cascades East, South',
           #'WA Cascades West, Central', 'WA Cascades West, Mt Baker', 'WA Cascades West, South'
           #]           
           'Canada': ["Northwest Coastal", "Northwest Inland", "Sea To Sky", 
                       "South Coast Inland", "South Coast", "North Rockies", 
                       "Cariboos", "North Columbia", "South Columbia", "Purcells", 
                       "Kootenay Boundary", "South Rockies", "Lizard Range and Flathead", 
                       "Vancouver Island", "Kananaskis Country, Alberta Parks", "Chic Chocs, Avalanche Quebec",
                       "Little Yoho", "Banff, Yoho and Kootenay National Parks", "Glacier National Park",
                       "Waterton Lakes National Park", "Jasper National Park"]
           }
seasons = ['15-16', '16-17', '17-18', '18-19', '19-20', '20-21']

In [4]:
#export
class ConvertToZarr:
    """
    Class which encapsulates the logic to convert a set of filtered netCDF files to Zarr
    """
    
    def __init__(self, seasons, regions, data_root, interpolate=1, resample_length='1d'):
        """
        Initialize the class
        
        Keyword Arguments
        seasons: list of season values to process
        regions: dictonary of Key: State and Value: List of Regions to process for that state
        data_root: the root path of the data folders which contains the 3.GFSFiltered1xInterpolation
        interpolate: the amount of interpolation applied in in the previous ParseGFS notebook (used for finding the correct input/output paths)
        """
        self.processed_path = data_root + '/3.GFSFiltered'+ str(interpolate) + 'xInterpolation' + resample_length + '/'
        self.zarr_base_path = output_root + '/4.GFSFiltered'+ str(interpolate) + 'xInterpolationZarr' + resample_length + '/'
        
        self.seasons = seasons
        self.regions = regions
        self.data_root = data_root
        self.compressor = Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE)
        
        if not os.path.exists(self.zarr_base_path):
            os.makedirs(self.zarr_base_path)
    
    def compute_region(self, region_name, season, state):
        """
        Calculates the zarr conversion for a specific region, season and state and indexes it for efficient lookup 
        
        Keyword Arguments
        region_name: name of the region to process
        season: season to process
        state: state to process (region must be a part of the state)
        """
        first = True
        base_path = self.processed_path + season + '/' + '/Region_' + region_name 
        zarr_path = self.zarr_base_path + season + '/' + state + '/Region_' + region_name + '.zarr'
        
        #TODO: refactor these to be shared code as logic also exists in ParseGFS
        p = 181
        if season in ['15-16', '19-20']:
            p = 182 #leap years
        
        if season == '15-16':
            snow_start_date = '2015-11-01'
        elif season == '16-17':
            snow_start_date = '2016-11-01'
        elif season == '17-18':
            snow_start_date = '2017-11-01'
        elif season == '18-19':
            snow_start_date = '2018-11-01'
        elif season == '19-20':
            snow_start_date = '2019-11-01'
        elif season == '20-21':
            snow_start_date = '2020-11-01'
        else:
            raise Exception('No known season ' + season)

        date_values_pd = pd.date_range(snow_start_date, periods=p, freq="D")
        try:
            with xr.open_zarr(zarr_path) as z:
                if z.time.values[-1] == date_values_pd[-1]:
                    print(' already exists: ' + region_name + ' ' + season + ' ' + state)
                    z.close()
                    return
                else:
                    #already exists but incomplete
                    date_values_pd = [pd.Timestamp(v) for v in date_values_pd.values.astype('datetime64[ns]') if v not in z.time.values]
                    print(' some exist but have to complete ' + str(len(date_values_pd)))
                    first = False
        except ValueError as err:
            #ignore as it doesn't exist yet
            pass
        
        #sometimes vars get added, filter to only the list of vars in the first dataset for that region
        #TODO: handle the case where the first dataset has more vars than subsequent ones
        final_vars = None
        last_ds = xr.open_dataset(base_path + '_' + date_values_pd[-1].strftime('%Y%m%d') + '.nc')
        #TODO: instead of variable-1, try vars -1
        last_ds = last_ds.to_array(name='vars').chunk({'time':-1, 'latitude':-1, 'longitude':-1, 'variable':-1}).to_dataset()
        last_vars = list(last_ds.variable.values)
        
        for d in date_values_pd:

            path =  base_path + '_' + d.strftime('%Y%m%d') + '.nc'
            print('On ' + str(path.split('/')[-1]))

            try:
                ds = xr.open_dataset(path, chunks={'latitude':1, 'longitude':1})
            except OSError as err:
                print(' missing file: ' + path)
                continue

            ds = ds.to_array(name='vars').chunk({'time':-1, 'latitude':-1, 'longitude':-1, 'variable':-1}).to_dataset()


            if first:
                #find intersection of the first and last variables to try to ensure
                #that we are using only the intersection
                final_vars = [v for v in list(ds.variable.values) if v in last_vars]
                ds = ds.sel(variable=ds.variable.isin(final_vars))
                enc = {x: {"compressor": self.compressor} for x in ds}
                ds.sortby('variable').to_zarr(zarr_path, consolidated=True, encoding=enc)
                first=False
            else:
                assert(final_vars is not None)
                ds = ds.sel(variable=ds.variable.isin(final_vars))                
                ds.sortby('variable').to_zarr(zarr_path, consolidated=True, append_dim='time')


    def process_tuple(self, t): 
        """
        Entry method to call compute_region with a tuple
        Basically a helper for executing with joblib parallel
        
        Keyword Arguments
        t: the tuple containing the region, season and state
        """
        self.compute_region(t[0], t[1], t[2])
    
    def make_list(self):
        """
        Helper method to make the list of values to process
        """
        to_process = []
        for s in self.seasons:
            for state in self.regions.keys():           
                for r in self.regions[state]:
                    to_process.append((r,s,state))
        return to_process
    
    def convert_local(self, jobs=15):
        l = self.make_list()
    
        #one state & season takes about 6 hours with 15 cores on my machine
        Parallel(n_jobs=jobs, backend="multiprocessing")(map(delayed(self.process_tuple), l))

In [5]:
#test_ignore
ctz = ConvertToZarr(seasons, regions, data_root, resample_length='3H')

In [6]:
%%time
#test_ignore
#1 region two seasons
#1 disk
#20 cores, no compression 
#CPU times: user 902 ms, sys: 472 ms, total: 1.37 s
#Wall time: 23min 36s
#103MB
#compression same time but 52MB

# all regions two seasons
# 2 disks
# 12 cores
# Wall Time 132 mins
# 1.2GB

#all regions(CO and CA) all  seasons
#2 disks
#20 cores
#CPU times: user 1min 48s, sys: 46.9 s, total: 2min 35s
#Wall time: 1d 12h 1min 33s
# 15gb for all data compressed

ctz.convert_local(jobs = 20)

On Region_South Columbia_20151101.nc
On Region_South Coast Inland_20151101.nc
On Region_North San Juan Mountains_20151101.nc
On Region_North Columbia_20151101.nc
On Region_Sawatch Zone_20151101.nc
On Region_Sangre de Cristo Range_20151101.nc
On Region_Cariboos_20151101.nc
On Region_Grand Mesa Zone_20151101.nc
On Region_Aspen Zone_20151101.nc
On Region_Steamboat Zone_20151101.nc
On Region_South Coast_20151101.nc
On Region_South San Juan Mountains_20151101.ncOn Region_Purcells_20151101.ncOn Region_Sea To Sky_20151101.nc

On Region_Vail Summit Zone_20151101.nc

On Region_Gunnison Zone_20151101.nc
On Region_Front Range Zone_20151101.nc
On Region_Northwest Coastal_20151101.ncOn Region_North Rockies_20151101.nc

On Region_Northwest Inland_20151101.nc
On Region_Aspen Zone_20151102.nc
On Region_Sawatch Zone_20151102.nc
On Region_Grand Mesa Zone_20151102.nc
On Region_Vail Summit Zone_20151102.nc
On Region_Gunnison Zone_20151102.nc
On Region_Aspen Zone_20151103.nc
On Region_Sawatch Zone_20151103

On Region_Sawatch Zone_20151202.nc
On Region_Sangre de Cristo Range_20151109.nc
On Region_South Columbia_20151105.nc
On Region_Gunnison Zone_20151119.nc
On Region_Grand Mesa Zone_20151203.nc
On Region_Aspen Zone_20151203.nc
On Region_Vail Summit Zone_20151130.nc
On Region_Sawatch Zone_20151203.nc
On Region_South San Juan Mountains_20151110.nc
On Region_Grand Mesa Zone_20151204.nc
On Region_Aspen Zone_20151204.nc
On Region_Steamboat Zone_20151108.nc
On Region_Sawatch Zone_20151204.nc
On Region_Vail Summit Zone_20151201.nc
On Region_Front Range Zone_20151107.nc
On Region_Northwest Coastal_20151102.nc
On Region_Gunnison Zone_20151120.nc
On Region_North San Juan Mountains_20151111.nc
On Region_Grand Mesa Zone_20151205.nc
On Region_Aspen Zone_20151205.nc
On Region_South Coast_20151112.nc
On Region_Sawatch Zone_20151205.nc
On Region_Vail Summit Zone_20151202.nc
On Region_Grand Mesa Zone_20151206.nc
On Region_Aspen Zone_20151206.nc
On Region_Sawatch Zone_20151206.nc
On Region_Vail Summit Zone

On Region_Vail Summit Zone_20151229.nc
On Region_South Coast Inland_20151106.nc
On Region_Cariboos_20151106.nc
On Region_North San Juan Mountains_20151120.nc
On Region_Grand Mesa Zone_20160105.nc
On Region_Aspen Zone_20160105.nc
On Region_South Coast_20151122.nc
On Region_Sawatch Zone_20160105.nc
On Region_South San Juan Mountains_20151119.nc
On Region_Vail Summit Zone_20151230.nc
On Region_Gunnison Zone_20151208.nc
On Region_Grand Mesa Zone_20160106.nc
On Region_Aspen Zone_20160106.nc
On Region_Sawatch Zone_20160106.nc
On Region_Steamboat Zone_20151115.nc
On Region_Vail Summit Zone_20151231.nc
On Region_Grand Mesa Zone_20160107.nc
On Region_Aspen Zone_20160107.nc
On Region_Sangre de Cristo Range_20151118.nc
On Region_Northwest Coastal_20151103.nc
On Region_Sawatch Zone_20160107.nc
On Region_Front Range Zone_20151113.nc
On Region_Gunnison Zone_20151209.nc
On Region_Vail Summit Zone_20160101.nc
On Region_Grand Mesa Zone_20160108.nc
On Region_Aspen Zone_20160108.nc
On Region_South Coast_

On Region_Sawatch Zone_20160206.nc
On Region_Vail Summit Zone_20160128.nc
On Region_Gunnison Zone_20151226.nc
On Region_Sea To Sky_20151111.nc
On Region_Grand Mesa Zone_20160207.nc
On Region_South San Juan Mountains_20151128.nc
On Region_Aspen Zone_20160207.nc
On Region_Sawatch Zone_20160207.nc
On Region_Vail Summit Zone_20160129.nc
On Region_Sangre de Cristo Range_20151126.nc
On Region_Grand Mesa Zone_20160208.nc
On Region_North San Juan Mountains_20151130.nc
On Region_Aspen Zone_20160208.nc
On Region_South Coast_20151203.nc
On Region_Gunnison Zone_20151227.nc
On Region_Steamboat Zone_20151122.nc
On Region_Sawatch Zone_20160208.nc
On Region_Purcells_20151111.nc
On Region_Vail Summit Zone_20160130.nc
On Region_Grand Mesa Zone_20160209.nc
On Region_Aspen Zone_20160209.nc
On Region_Sawatch Zone_20160209.nc
On Region_Northwest Coastal_20151104.nc
On Region_North Rockies_20151105.nc
On Region_Vail Summit Zone_20160131.nc
On Region_Grand Mesa Zone_20160210.nc
On Region_Front Range Zone_2015

On Region_Cariboos_20151111.nc
On Region_Grand Mesa Zone_20160310.nc
On Region_Aspen Zone_20160310.nc
On Region_South Coast_20151213.nc
On Region_Sawatch Zone_20160310.nc
On Region_Sangre de Cristo Range_20151204.nc
On Region_South San Juan Mountains_20151207.nc
On Region_Vail Summit Zone_20160227.nc
On Region_North Columbia_20151112.nc
On Region_Grand Mesa Zone_20160311.nc
On Region_Aspen Zone_20160311.nc
On Region_Gunnison Zone_20160114.nc
On Region_Sawatch Zone_20160311.nc
On Region_Vail Summit Zone_20160228.nc
On Region_Grand Mesa Zone_20160312.nc
On Region_Aspen Zone_20160312.nc
On Region_Steamboat Zone_20151129.nc
On Region_Sawatch Zone_20160312.nc
On Region_Grand Mesa Zone_20160313.nc
On Region_Aspen Zone_20160313.nc
On Region_Vail Summit Zone_20160229.nc
On Region_Gunnison Zone_20160115.nc
On Region_North San Juan Mountains_20151210.nc
On Region_South Coast_20151214.nc
On Region_Sawatch Zone_20160313.nc
On Region_Grand Mesa Zone_20160314.nc
On Region_Aspen Zone_20160314.nc
On R

...

On Region_South Columbia_20210407.nc
On Region_Cariboos_20210223.nc
On Region_Sea To Sky_20210407.nc
On Region_Vancouver Island_20210128.nc
On Region_Kootenay Boundary_20210323.nc
On Region_Northwest Inland_20200302.nc
On Region_Purcells_20210228.nc
On Region_Northwest Coastal_20201226.nc
On Region_South Coast Inland_20210302.nc
On Region_North Columbia_20210222.nc
On Region_Northwest Inland_20201229.nc
On Region_South Columbia_20210408.nc
On Region_Vancouver Island_20210129.nc
On Region_Cariboos_20210224.nc
On Region_Sea To Sky_20210408.nc
On Region_Kootenay Boundary_20210324.nc
On Region_North Rockies_20201230.nc
On Region_North Rockies_20200305.nc
On Region_Purcells_20210301.nc
On Region_Northwest Coastal_20200213.nc
On Region_South Columbia_20210409.nc
On Region_North Columbia_20210223.nc
On Region_South Coast Inland_20210303.nc
On Region_Vancouver Island_20210130.nc
On Region_Kootenay Boundary_20210325.nc
On Region_Purcells_20210302.nc
On Region_Sea To Sky_20210409.nc
On Region_Ca

On Region_Kootenay Boundary_20210415.nc
On Region_North Rockies_20210108.nc
On Region_North Rockies_20200314.nc
On Region_Cariboos_20210314.nc
On Region_Vancouver Island_20210220.nc
On Region_North Columbia_20210314.nc
On Region_Northwest Coastal_20200220.nc
On Region_Purcells_20210324.nc
On Region_Kootenay Boundary_20210416.nc
On Region_South Coast Inland_20210321.nc
On Region_Sea To Sky_20210427.nc
On Region_Vancouver Island_20210221.nc
On Region_Cariboos_20210315.nc
On Region_Northwest Inland_20200311.nc
On Region_North Columbia_20210315.nc
On Region_Purcells_20210325.nc
On Region_Northwest Inland_20210107.nc
On Region_Kootenay Boundary_20210417.nc
On Region_Sea To Sky_20210428.nc
On Region_South Coast Inland_20210322.nc
On Region_Vancouver Island_20210222.nc
On Region_North Rockies_20210109.nc
On Region_North Rockies_20200315.nc
On Region_Cariboos_20210316.nc
On Region_North Columbia_20210316.nc
On Region_Kootenay Boundary_20210418.nc
On Region_Purcells_20210326.nc
On Region_Northw

On Region_Purcells_20210420.nc
On Region_Cariboos_20210406.nc
On Region_Northwest Inland_20200323.nc
On Region_South Coast Inland_20210411.nc
On Region_North Rockies_20200328.nc
On Region_North Columbia_20210408.nc
On Region_North Rockies_20210122.nc
On Region_Northwest Inland_20210119.nc
On Region_Vancouver Island_20210325.nc
On Region_Purcells_20210421.nc
On Region_Northwest Coastal_20210113.nc
On Region_Cariboos_20210407.nc
On Region_South Coast Inland_20210412.nc
On Region_Northwest Coastal_20200302.nc
On Region_North Columbia_20210409.nc
On Region_Vancouver Island_20210326.nc
On Region_Purcells_20210422.nc
On Region_North Rockies_20200329.nc
On Region_Northwest Inland_20200324.nc
On Region_North Rockies_20210123.nc
On Region_Northwest Inland_20210120.nc
On Region_Cariboos_20210408.nc
On Region_Vancouver Island_20210327.nc
On Region_Purcells_20210423.nc
On Region_North Columbia_20210410.nc
On Region_South Coast Inland_20210413.nc
On Region_Northwest Coastal_20210114.nc
On Region_Pu

On Region_Northwest Inland_20210205.nc
On Region_Northwest Coastal_20200316.nc
On Region_Vancouver Island_20210430.nc
On Region_Northwest Inland_20200410.nc
On Region_North Rockies_20200417.nc
On Region_North Rockies_20210211.nc
On Region_Northwest Inland_20210206.nc
On Region_Northwest Coastal_20210128.nc
On Region_Northwest Coastal_20200317.nc
On Region_North Rockies_20200418.nc
On Region_North Rockies_20210212.nc
On Region_Northwest Inland_20200411.nc
On Region_Northwest Inland_20210207.nc
On Region_Northwest Coastal_20210129.nc
On Region_Northwest Coastal_20200318.nc
On Region_North Rockies_20200419.nc
On Region_North Rockies_20210213.nc
On Region_Northwest Inland_20200412.nc
On Region_Northwest Inland_20210208.nc
On Region_North Rockies_20200420.nc
On Region_North Rockies_20210214.nc
On Region_Northwest Coastal_20210130.nc
On Region_Northwest Coastal_20200319.nc
On Region_Northwest Inland_20200413.nc
On Region_Northwest Inland_20210209.nc
On Region_North Rockies_20200421.nc
On Reg

On Region_Northwest Coastal_20210308.nc
On Region_Northwest Inland_20210324.nc
On Region_Northwest Coastal_20200425.nc
On Region_North Rockies_20210402.nc
On Region_Northwest Coastal_20210309.nc
On Region_Northwest Coastal_20200426.nc
On Region_Northwest Inland_20210325.nc
On Region_North Rockies_20210403.nc
On Region_Northwest Coastal_20210310.nc
On Region_Northwest Coastal_20200427.nc
On Region_Northwest Inland_20210326.nc
On Region_North Rockies_20210404.nc
On Region_Northwest Coastal_20210311.nc
On Region_Northwest Coastal_20200428.nc
On Region_Northwest Inland_20210327.nc
On Region_North Rockies_20210405.nc
On Region_Northwest Coastal_20210312.nc
On Region_Northwest Coastal_20200429.nc
On Region_Northwest Inland_20210328.nc
On Region_North Rockies_20210406.nc
On Region_Northwest Coastal_20210313.nc
On Region_Northwest Coastal_20200430.nc
On Region_North Rockies_20210407.nc
On Region_Northwest Inland_20210329.nc
On Region_Northwest Coastal_20210314.nc
On Region_North Rockies_202104